# Day 22: Reactor Reboot

https://adventofcode.com/2021/day/22

## Part 1

**how many cubes are on?**

In [1]:
from IPython.display import Markdown
import numpy as np
import itertools

In [95]:
#infile = 'test_input1.txt'
#infile = 'test_input2.txt'
infile = 'input.txt'

def get_tup(dat):
    nums = dat.split('=')[-1]
    return tuple(int(x) for x in nums.split('..'))

boundlist = list()
xmin = None; ymin = None; zmin = None
xmax = None; ymax = None; zmax = None
limit = 50
with open(infile, 'r') as fid:
    for line in fid:
        state, bounds = line.strip().split()
        x, y, z = bounds.split(',')
        xtup = get_tup(x)
        ytup = get_tup(y)
        ztup = get_tup(z)
        # Ignore anything outside of +/-limit
        if np.any(np.array(xtup) < -limit) or np.any(np.array(xtup) > limit):
            continue
        if np.any(np.array(ytup) < -limit) or np.any(np.array(ytup) > limit):
            continue
        if np.any(np.array(ztup) < -limit) or np.any(np.array(ztup) > limit):
            continue
        boundlist.append([state, xtup, ytup, ztup])
        xmin = min(xmin, xtup[0]) if xmin is not None else xtup[0]
        ymin = min(ymin, ytup[0]) if ymin is not None else ytup[0]
        zmin = min(zmin, ztup[0]) if zmin is not None else ztup[0]
        xmax = max(xmax, xtup[1]) if xmax is not None else xtup[1]
        ymax = max(ymax, ytup[1]) if ymax is not None else ytup[1]
        zmax = max(zmax, ztup[1]) if zmax is not None else ztup[1]
#boundlist

In [96]:
#reactor_size = 102
#reactor_size = 14
#reactor = np.zeros((reactor_size, reactor_size, reactor_size), dtype=np.bool)
reactor = np.zeros((zmax-zmin+1, ymax-ymin+1, xmax-xmin+1), dtype=np.bool)
reactor.shape

(93, 95, 94)

In [97]:
for state, xb, yb, zb in boundlist:
    x0, x1 = xb
    y0, y1 = yb
    z0, z1 = zb
    # x coord -> k (cols)
    # y coord -> j (rows)
    # z coord -> i (depth)
    # Convert x,y,z to i,j,k by making zero indexed
    i0, i1 = (z0 - zmin, z1 - zmin)
    j0, j1 = (y0 - ymin, y1 - ymin)
    k0, k1 = (x0 - xmin, x1 - xmin)
    rslice = reactor[i0:i1+1,j0:j1+1,k0:k1+1]
    if 'on' == state:
        # Turn on cubes with logical or
        kern = np.ones((i1-i0+1, j1-j0+1, k1-k0+1), dtype=np.bool)
        reactor[i0:i1+1,j0:j1+1,k0:k1+1] = np.logical_or(rslice, kern)
    if 'off' == state:
        # Turn off cubes with logical and
        kern = np.zeros((i1-i0+1, j1-j0+1, k1-k0+1), dtype=np.bool)
        reactor[i0:i1+1,j0:j1+1,k0:k1+1] = np.logical_and(rslice, kern)

oncubes = reactor.sum()
Markdown("The number of 'on' cubes is **{}**".format(oncubes))

The number of 'on' cubes is **596989**

## Part Two

**how many cubes are on?**

In [1]:
from IPython.display import Markdown
import numpy as np
import itertools

In [4]:
#infile = 'test_input1.txt'
infile = 'test_input2.txt'
#infile = 'test_input3.txt'
#infile = 'input.txt'

def get_tup(dat):
    nums = dat.split('=')[-1]
    return tuple(int(x) for x in nums.split('..'))

boundlist = list()
#limit = 50
limit = None
with open(infile, 'r') as fid:
    for line in fid:
        state, bounds = line.strip().split()
        x, y, z = bounds.split(',')
        xtup = get_tup(x)
        ytup = get_tup(y)
        ztup = get_tup(z)
        if limit is not None:
            # Ignore anything outside of +/-limit
            if np.any(np.array(xtup) < -limit) or np.any(np.array(xtup) > limit):
                continue
            if np.any(np.array(ytup) < -limit) or np.any(np.array(ytup) > limit):
                continue
            if np.any(np.array(ztup) < -limit) or np.any(np.array(ztup) > limit):
                continue
        boundlist.append([state, xtup, ytup, ztup])
#boundlist

In [ ]:
# Represent N-dimensional data as dictionary of keys
# If key does not exist in dictionary, assume value there is False
# If key does exists, then we can get it's value directly
# We can also consider "turning off" by deleting the key from the dictionary

def get_keys(xtup, ytup, ztup):
    # From supplied x/y/z low to high tuples,
    # generate out all x,y,z tuples for the cuboid
    ii = itertools.product(range(xtup[0], xtup[1] + 1),
                           range(ytup[0], ytup[1] + 1),
                           range(ztup[0], ztup[1] + 1))
    return set(ii)

# Initialize reactor as empty set
reactor = set()
niters = len(boundlist)
i = 1
for state, xb, yb, zb in boundlist:
    print('{}/{}...'.format(i, niters), end='')
    ctups = get_keys(xb, yb, zb)
    if 'on' == state:
        # Add keys to set
        reactor |= ctups
    else:
        # Remove keys from set
        reactor -= ctups
    i+=1
    
oncubes = len(reactor)
Markdown("The number of 'on' cubes is **{}**".format(oncubes))

1/22...2/22...3/22...4/22...5/22...6/22...7/22...

In [5]:
'''
# Initialize reactor as empty dictionary
reactor = dict()

niters = len(boundlist)
i = 1
for state, xb, yb, zb in boundlist:
    print('{}/{}...'.format(i, niters), end='')
    ctups = get_keys(xb, yb, zb)
    if 'on' == state:
        # Turn on all keys
        for tup in ctups:
            reactor[tup] = 1
    if 'off' == state:
        # Delete any matching keys that are "on"
        keykill = ctups & reactor.keys()
        for tup in keykill:
            reactor.pop(tup)
    i += 1

oncubes = sum(reactor.values())
'''
#Markdown("The number of 'on' cubes is **{}**".format(oncubes))

1/60...2/60...3/60...4/60...5/60...6/60...7/60...8/60...9/60...10/60...11/60...12/60...13/60...14/60...15/60...16/60...17/60...18/60...19/60...20/60...21/60...22/60...23/60...24/60...25/60...26/60...27/60...28/60...29/60...30/60...31/60...32/60...33/60...34/60...35/60...36/60...37/60...38/60...39/60...40/60...41/60...42/60...43/60...44/60...45/60...46/60...47/60...48/60...49/60...50/60...51/60...52/60...53/60...54/60...55/60...56/60...57/60...58/60...59/60...60/60...

'\n# Initialize reactor as empty dictionary\nreactor = dict()\n\nniters = len(boundlist)\ni = 1\nfor state, xb, yb, zb in boundlist:\n    print(\'{}/{}...\'.format(i, niters), end=\'\')\n    ctups = get_keys(xb, yb, zb)\n    if \'on\' == state:\n        # Turn on all keys\n        for tup in ctups:\n            reactor[tup] = 1\n    if \'off\' == state:\n        # Delete any matching keys that are "on"\n        keykill = ctups & reactor.keys()\n        for tup in keykill:\n            reactor.pop(tup)\n    i += 1\n\noncubes = sum(reactor.values())\n'

In [1]:
# Just indexing only the on cubes is too much memory/overhead
# Probably need to represent only cuboid boundaries to be broken
# apart logically into sub-cuboids as needed. Then the volume of
# the remaining cuboid boundaries will be the answer

# Should be straightforward to solve with a constructive solid geometry
# library or openscad